# Your First RAG Application: Personal Wellness Assistant

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application by building a **Personal Wellness Assistant**.

Imagine having an AI assistant that can answer your health and wellness questions based on a curated knowledge base - that's exactly what we'll build here. We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents (Loading our Wellness Knowledge Base)
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation (Building the Wellness Assistant)
  - 🚧 Activity #1: Enhance Your Wellness Assistant

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utilities

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using a comprehensive Health & Wellness Guide as our sample data. This content covers exercise, nutrition, sleep, stress management, and healthy habits - perfect for building a personal wellness assistant!

In [3]:
text_loader = TextFileLoader("data/HealthWellnessGuide.txt")
documents = text_loader.load_documents()
len(documents)

1

In [4]:
print(documents[0][:100])

The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into manageable sized chunks that retain the most relevant local context.

In [5]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

21

Let's take a look at some of the documents we've managed to split.

In [6]:
split_documents[0:1]

['The Personal Wellness Guide\nA Comprehensive Resource for Health and Well-being\n\nPART 1: EXERCISE AND MOVEMENT\n\nChapter 1: Understanding Exercise Basics\n\nExercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.\n\nThe four main types of exercise are aerobic (cardio), strength training, flexibility, and balance exercises. A well-rounded fitness routine includes all four types. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, along with muscle-strengthening activities on 2 or more days per week.\n\nChapter 2: Exercises for Common Problems\n\nLower Back Pain Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [7]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this paper](https://arxiv.org/abs/2205.13147) for an answer to question #1.2!


##### ✅ Answer:
1. Yes. You can reduce the embedding dimension by setting the dimensions parameter in the API call (1-1536 range). This saves storage and speeds up search, but you cannot exceed the model's native 1536 dimensions.

2. OpenAI uses Matryoshka Representation Learning, training the model so the first N dimensions capture the most semantic value. Truncating the tail dimensions maintains quality while reducing vector size.


We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
Non-blocking I/O efficiency: Async allows concurrent API calls to OpenAI during embedding generation (network-bound), overlapping requests instead of sequential waits—up to 10x throughput for large batches.
Higher concurrency: Process thousands of texts simultaneously without threads, ideal for RAG pipelines or real-time apps, reducing total latency.
Scalability in async frameworks: Integrates seamlessly with FastAPI/Starlette event loops, handling more requests/sec while keeping low memory/CPU usage.


So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [9]:
vector_db.search_by_text("What exercises help with lower back pain?", k=3)

[(' Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes.\n\nRecommended exercises for lower back pain include:\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs an

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicity's sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analogous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
Yes. Embeddings from text-embedding-3-small are fully deterministic—identical inputs always produce identical vectors, no special parameters needed. For chat/completions, set seed parameter + temperature=0 for highly reproducible outputs (though not 100% guaranteed due to potential backend changes).

### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [10]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [11]:
print(response)

Hello! The best way to write a loop in Python depends on what you want to achieve, but generally, a `for` loop is very clean and Pythonic when iterating over sequences like lists, strings, or ranges.

Here’s a simple example of a `for` loop that prints numbers from 0 to 4:

```python
for i in range(5):
    print(i)
```

If you need to loop while some condition is true, a `while` loop is appropriate:

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

If you share what kind of loop or task you want to perform, I’d be happy to help you with a more specific example! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [13]:
RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [14]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [15]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What are some natural remedies for improving sleep quality?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: Based on the provided context, some natural remedies for improving sleep quality include:

1. Cognitive Behavioral Therapy for Insomnia (CBT-I) – a structured approach to addressing insomnia (Source 1).
2. Relaxation techniques such as progressive muscle relaxation and meditation (Sources 1 and 3).
3. Herbal teas like chamomile or valerian root (Source 1).
4. Magnesium supplements, but only after consulting a healthcare provider (Source 1).
5. Creating an optimal sleep environment—keeping bedroom temperature between 65-68°F (18-20°C), using blackout curtains or sleep masks for darkness, and ensuring quiet with white noise machines or earplugs (Sources 1, 2, and 3).
6. Establishing good sleep hygiene practices including:
   - Avoiding caffeine after 2 PM
   - Limiting alcohol and heavy meals before bedtime
   - Maintaining a consistent sleep schedule
   - Limiting screen exposure 1-2 hours before bed
   - Engaging in relaxing bedtime routines such as reading, gentle stretching

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
Chain-of-Thought (CoT) prompting: Explicitly instruct the LLM to "think step-by-step" or provide few-shot examples showing detailed reasoning chains (e.g., "Step 1: Analyze X, Step 2: Consider Y..."). This elicits more thoughtful, accurate responses by simulating human-like intermediate logic.

### 🏗️ Activity #1:

Enhance your Personal Wellness Assistant in some way! 

Suggestions are: 

- **PDF Support**: Allow it to ingest wellness PDFs (meal plans, workout guides, medical information sheets)
- **New Distance Metric**: Implement a different similarity measure - does it improve retrieval for health queries?
- **Metadata Support**: Add metadata like topic categories (exercise, nutrition, sleep) or difficulty levels to help filter results
- **Different Embedding Model**: Try a different embedding model - does domain-specific tuning help for health content?
- **Multi-Source Ingestion**: Add the capability to ingest content from YouTube health videos, podcasts, or health blogs

While these are suggestions, you should feel free to make whatever augmentations you desire! Think about what features would make this wellness assistant most useful for your personal health journey.

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [34]:
### YOUR CODE HERE
import pandas as pd
 
csv_path = "data/Meal_plans.csv"
 
df = pd.read_csv(csv_path)
 
print(f"CSV loaded with {len(df)} rows")
df.head(5)

CSV loaded with 33 rows


,day,meal,recipe,ingredients,calories,protein_g,fat_g,carbs_g,prep_time_min,dietary_label
0,Monday,Breakfast,Greek Yogurt Parfait,"Greek yogurt (200g), mixed berries (100g), alm...",420,28,18,32,5,High Protein
1,Monday,Morning Snack,Apple + Peanut Butter,"Apple (medium), peanut butter (1 tbsp)",220,6,16,20,2,Balanced
2,Monday,Lunch,Chicken Quinoa Salad,"Grilled chicken (150g), quinoa (100g cooked), ...",650,45,22,48,15,High Protein
3,Monday,Afternoon Snack,Carrot Sticks + Hummus,"Carrots (150g), hummus (50g)",180,5,10,22,3,Plant-Based
4,Monday,Dinner,Baked Salmon + Veggies,"Salmon fillet (150g), broccoli (200g), sweet p...",680,42,32,45,25,Omega-3 Rich


In [35]:
csv_texts = []
 
for _, row in df.iterrows():
    doc_text = (
        f"SourceType: CSV\n"
        f"Day: {row.get('day','')}\n"
        f"Meal: {row.get('meal','')}\n"
        f"Recipe: {row.get('recipe','')}\n"
        f"Ingredients: {row.get('ingredients','')}\n"
        f"Calories: {row.get('calories','')}\n"
        f"Protein_g: {row.get('protein_g','')}\n"
        f"Fat_g: {row.get('fat_g','')}\n"
        f"Carbs_g: {row.get('carbs_g','')}\n"
        f"Prep_Time_min: {row.get('prep_time_min','')}\n"
        f"Dietary_Label: {row.get('dietary_label','')}"
    )
    csv_texts.append(doc_text)
 
print(f"Prepared {len(csv_texts)} documents")
 
print("\nSample document:\n")
print(csv_texts[0][:1000])

Prepared 33 documents

Sample document:

SourceType: CSV
Day: Monday
Meal: Breakfast
Recipe: Greek Yogurt Parfait
Ingredients: Greek yogurt (200g), mixed berries (100g), almonds (20g)
Calories: 420
Protein_g: 28
Fat_g: 18
Carbs_g: 32
Prep_Time_min: 5
Dietary_Label: High Protein


In [36]:
import asyncio
 
csv_vectors = await vector_db.embedding_model.async_get_embeddings(csv_texts)
 
for text, vec in zip(csv_texts, csv_vectors):
    vector_db.insert(text, vec)
 
print(f"Inserted {len(csv_texts)} documents into vector_db")
 

Inserted 33 documents into vector_db


In [37]:
# Recreate the baseline (text-only) vector database using the same library primitives
 
vector_db_baseline = VectorDatabase()
 
# Create embeddings for the original text chunks (batch)
baseline_vectors = await vector_db_baseline.embedding_model.async_get_embeddings(split_documents)
 
# Insert text + vector pairs
for text, vec in zip(split_documents, baseline_vectors):
    vector_db_baseline.insert(text, vec)
 
print(f"Baseline vector database rebuilt (text-only) with {len(split_documents)} chunks")
 

Baseline vector database rebuilt (text-only) with 21 chunks


In [38]:
query = "What is your recommendation for a meal plan for a beginner?"
 
# baseline:
baseline_results = vector_db_baseline.search_by_text(query, k=7)
 
print("\n--- Baseline (text-only) retrieval ---")
for i, (ctx, score) in enumerate(baseline_results, 1):
    print(f"{i}) score={float(score):.3f} | {ctx[:160].replace('\n',' ')}...")
 
# augmented:
augmented_results = vector_db.search_by_text(query, k=7)
 
print("\n--- Augmented (text + CSV) retrieval ---")
for i, (ctx, score) in enumerate(augmented_results, 1):
    print(f"{i}) score={float(score):.3f} | {ctx[:160].replace('\n',' ')}...")
 


--- Baseline (text-only) retrieval ---
1) score=0.458 | ls of Healthy Eating  A balanced diet provides your body with the nutrients it needs to function properly. The key components include:  Macronutrients: - Carboh...
2) score=0.456 |  repeat 10 times.  Chapter 3: Building a Workout Routine  Starting a new exercise routine can feel overwhelming. The key is to start slowly and gradually increa...
3) score=0.455 | l planning saves time, reduces stress, and helps maintain a healthy diet.  Weekly Meal Planning Steps: 1. Review your schedule for the week and identify busy da...
4) score=0.271 | ion  Habits are behaviors that become automatic through repetition. Understanding how habits form can help you build positive routines and break negative ones. ...
5) score=0.266 | chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs. - Pelvic Tilts: Lie on your back with knees bent, flatten your back against t...
6) score=0.264 | The Personal Wellness Guide A Comprehe

In [39]:
query = "What is best to eat on weekdays morning"
 
# baseline:
baseline_results = vector_db_baseline.search_by_text(query, k=7)
 
print("\n--- Baseline (text-only) retrieval ---")
for i, (ctx, score) in enumerate(baseline_results, 1):
    print(f"{i}) score={float(score):.3f} | {ctx[:160].replace('\n',' ')}...")
 
# augmented:
augmented_results = vector_db.search_by_text(query, k=7)
 
print("\n--- Augmented (text + CSV) retrieval ---")
for i, (ctx, score) in enumerate(augmented_results, 1):
    print(f"{i}) score={float(score):.3f} | {ctx[:160].replace('\n',' ')}...")


--- Baseline (text-only) retrieval ---
1) score=0.436 | ls of Healthy Eating  A balanced diet provides your body with the nutrients it needs to function properly. The key components include:  Macronutrients: - Carboh...
2) score=0.430 | l planning saves time, reduces stress, and helps maintain a healthy diet.  Weekly Meal Planning Steps: 1. Review your schedule for the week and identify busy da...
3) score=0.347 |  repeat 10 times.  Chapter 3: Building a Workout Routine  Starting a new exercise routine can feel overwhelming. The key is to start slowly and gradually increa...
4) score=0.328 | ke at a consistent time - Drink a glass of water immediately - 5-10 minutes of stretching or light movement - Healthy breakfast - Brief mindfulness practice or ...
5) score=0.309 | ion  Habits are behaviors that become automatic through repetition. Understanding how habits form can help you build positive routines and break negative ones. ...
6) score=0.308 | cklist: - 8 glasses of water - 5+ serv